In [4]:
import json
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

def reduce_memory(df):
    for col in df.columns:
        if df[col].dtype == 'float64':
            df[col] = df[col].astype('float32')
        if df[col].dtype == 'int64':
            df[col] = df[col].astype('int32')
    return df

df_games = reduce_memory(pd.read_csv("games.csv", index_col = "app_id"))
df_games["date_release"] = df_games["date_release"].astype(str).str[:4].astype(int)
df_games.rename(columns={"date_release": "year_release"}, inplace=True)
df_games.head(5)

,title,year_release,win,mac,linux,rating,positive_ratio,user_reviews,price_final,price_original,discount,steam_deck
app_id,,,,,,,,,,,,
13500,Prince of Persia: Warrior Within™,2008,True,False,False,Very Positive,84,2199,9.99,9.99,0.0,True
22364,BRINK: Agents of Change,2011,True,False,False,Positive,85,21,2.99,2.99,0.0,True
113020,Monaco: What's Yours Is Mine,2013,True,True,True,Very Positive,92,3722,14.99,14.99,0.0,True
226560,Escape Dead Island,2014,True,False,False,Mixed,61,873,14.99,14.99,0.0,True
249050,Dungeon of the ENDLESS™,2014,True,True,False,Very Positive,88,8784,11.99,11.99,0.0,True


In [6]:
file_path = "recommendations.csv"

# Read in chunks
chunksize = 10000  # Adjust based on memory
chunks = []

for chunk in pd.read_csv(file_path, index_col="app_id", chunksize=chunksize):
    chunk = reduce_memory(chunk)  # Apply memory reduction
    chunks.append(chunk)

# Combine all chunks into a single DataFrame
df_recs = pd.concat(chunks, ignore_index=False)

# Print result
df_recs.head()

,helpful,funny,date,is_recommended,hours,user_id,review_id
app_id,,,,,,,
975370,0,0,2022-12-12,True,36.299999,51580,0
304390,4,0,2017-02-17,False,11.500000,2586,1
1085660,2,0,2019-11-17,True,336.500000,253880,2
703080,0,0,2022-09-23,True,27.400000,259432,3
526870,0,0,2021-01-10,True,7.900000,23869,4


In [6]:
df_users = reduce_memory(pd.read_csv("users.csv", index_col = "user_id"))
df_users.head(5)

,products,reviews
user_id,,
7360263,359,0
14020781,156,1
8762579,329,4
4820647,176,4
5167327,98,2


In [6]:
meta_data = reduce_memory(pd.read_json("games_metadata.json"))
meta_data["tags"] = meta_data["tags"].apply(lambda x: ", ".join(x) if isinstance(x, list) else x)

meta_data.head(5)

,app_id,description,tags
0,13500,Enter the dark underworld of Prince of Persia ...,"Action, Adventure, Parkour, Third Person, Grea..."
1,22364,,Action
2,113020,Monaco: What's Yours Is Mine is a single playe...,"Co-op, Stealth, Indie, Heist, Local Co-Op, Str..."
3,226560,Escape Dead Island is a Survival-Mystery adven...,"Zombies, Adventure, Survival, Action, Third Pe..."
4,249050,Dungeon of the Endless is a Rogue-Like Dungeon...,"Roguelike, Strategy, Tower Defense, Pixel Grap..."


For analysing pricing trends we can find the average price of games by year of release:

In [11]:
avg_price = df_games.groupby("year_release")["price_final"].agg("mean")
print(avg_price)

year_release
1997     4.995000
1998     9.990000
1999     3.326667
2000     7.490000
2001     4.990000
2002     4.940000
2003     5.990000
2004     8.740000
2005     1.160000
2006     6.901786
2007     7.463658
2008     8.967465
2009     9.501024
2010     8.366302
2011     8.433165
2012     9.510566
2013     9.757214
2014     9.033061
2015     7.833925
2016     7.571791
2017     7.676552
2018     7.615424
2019     8.339868
2020     8.459524
2021     8.439894
2022     9.917744
2023    11.239883
Name: price_final, dtype: float32


Find the most common genres in the dataset to see which genres are trending

In [14]:
genres = []
genre_count = {}
#Many genres are listed per entry
#Split and count the occurences of each genre avoiding duplicates
for g in meta_data["tags"]:
    if pd.isna(g) or g.strip() == "":  
        continue
    for genre in g.split(","):
        genre = genre.strip()
        if genre not in genres:
            genres.append(genre)
            genre_count[genre] = 0
        
        genre_count[genre] +=1

sorted_genre_count = dict(sorted(genre_count.items(), key=lambda item: item[1], reverse=True))

for genre, count in sorted_genre_count.items():
    print(f"{genre}: {count}")

Indie: 27957
Singleplayer: 22566
Action: 21897
Adventure: 20183
Casual: 17844
Simulation: 12691
2D: 11438
Strategy: 11093
RPG: 10156
Atmospheric: 8688
Puzzle: 7585
Story Rich: 7149
Multiplayer: 6575
3D: 6142
Pixel Graphics: 6028
Exploration: 5769
Cute: 5702
Colorful: 5693
Free to Play: 5645
First-Person: 5570
Early Access: 5308
Fantasy: 5174
Anime: 5002
Funny: 4956
Horror: 4789
Arcade: 4502
Shooter: 4389
Family Friendly: 4286
Female Protagonist: 4222
Retro: 4153
Sci-fi: 4113
Platformer: 3998
Difficult: 3923
Relaxing: 3922
Great Soundtrack: 3912
Violent: 3676
Open World: 3603
Action-Adventure: 3367
Survival: 3265
Visual Novel: 3241
Co-op: 3214
Gore: 3185
Third Person: 3174
Comedy: 3125
VR: 3098
FPS: 3044
Psychological Horror: 2879
Stylized: 2874
Realistic: 2838
Point & Click: 2773
Controller: 2758
Sandbox: 2739
Mystery: 2737
Choices Matter: 2641
Top-Down: 2603
Dark: 2570
Sports: 2542
PvP: 2538
Physics: 2519
Combat: 2387
Multiple Endings: 2357
Minimalist: 2322
Sexual Content: 2294
Nudity

We could identify which price points generate the most recommendations:

In [ ]:
df_merged = df_games.merge(df_recs, on="app_id", how="inner")
df_price_recommendations =df_merged[df_merged["is_recommended"] == True].groupby("price_final")["app_id"].count().reset_index().rename(columns={"app_id": "num_recommendations"})

df_price_recommendations = df_price_recommendations.sort_values(by="num_recommendations", ascending=False)

# Display results
print(df_price_recommendations.head())